# **NLP for Telco Data (logs) Anonymization**


To anonymize Personally Identifiable Information (PII) from log files, the first step is to identifying PII.
This can be achieved through the utilization of various methods, such as:


*   1. Regular Expressions (Regex)
*   2. Presidio Lib
*   3. NLP Model
*   4. Deep Learning (Custom NLP Model)



**1.   Regular Expressions (Regex):**



In [13]:
import re

# Defining regular expressions for different types of PII
ip_pattern = r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b'
username_pattern = r'\buser\s+\w+\b'
email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
userid_pattern = r'\buser(ID)?\s+\d+\b'

# Reading the log file
with open('/content/Linux_2k.log', 'r') as file:
    log_data = file.read()

# Anonymizing IP addresses
log_data_anonymized = re.sub(ip_pattern, 'xxx.xxx.xxx.xxx', log_data)

# Anonymizing usernames
log_data_anonymized = re.sub(username_pattern, 'user XXX', log_data_anonymized)

# Anonymizing email addresses
log_data_anonymized = re.sub(email_pattern, 'email@example.com', log_data_anonymized)

# Anonymizing user IDs
log_data_anonymized = re.sub(userid_pattern, 'user ID XXX', log_data_anonymized)

with open('re_anonymized.txt', 'w') as file:
    file.write(log_data_anonymized)



Limitations of regex-based PII identification:

*   Limited accuracy due to inability to capture all variations of PII.
*   Lack of context sensitivity, leading to false positives or negatives.
*   Difficulty in maintenance and handling variability in PII formats.
*   Inability to handle nested structures and overlook encrypted data.

**2. Presidio Lib:**

In [14]:
!pip install presidio-analyzer presidio-anonymizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 67.5 MB/s eta 0:00:00


In [15]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine

# Read log file
with open("/content/Linux_2k.log", "r") as file:
    text = file.read()


# Initialize the analyzer engine
analyzer = AnalyzerEngine()

# Analyze text for PII
results = analyzer.analyze(text=text, entities=[], language='en')

# Initialize the anonymizer engine
anonymizer = AnonymizerEngine()

# Anonymize PII in text
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=results)

# Get the anonymized text from the result
#anonymized_text = anonymized_result.anonymized_text

# Write anonymized data to a new file
with open("Presidio_anonymized.txt", "w") as output_file:
    output_file.write(anonymized_result.text)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Limitations of Presidio-based PII identification:

*   Context Sensitivity: Presidio may struggle with context-dependent PII, such as ambiguous terms like "address" or "credit", leading to potential false positives or negatives.
*   Customization Complexity: Tailoring Presidio to specific data types or domains requires considerable effort, potentially hindering its adaptability in diverse environments.
*   Performance Overhead: Analyzing large datasets with Presidio could incur significant computational overhead due to its rule-based and machine learning-based processing.
*   Language Dependency: Presidio's effectiveness may vary across languages, as it relies on language-specific models and dictionaries, limiting its applicability in multilingual contexts.



**3. NLP Model:**

In [16]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Define a function to anonymize PII
def anonymize_pii(text):
    doc = nlp(text)
    anonymized_text = []
    for token in doc:
        if token.ent_type_ in ["PERSON", "GPE", "ORG", "CARDINAL", "NORP", "FAC"]:
            anonymized_text.append(token.ent_type_)
        else:
            anonymized_text.append(token.text)
    return " ".join(anonymized_text)

# Read the log file
with open("/content/Linux_2k.log", "r") as file:
    text = file.read()

# Anonymize PII in the text
anonymized_text = anonymize_pii(text)

# Write anonymized data to a new file
with open("NLP_anonymized.txt", "w") as output_file:
    output_file.write(anonymized_text)


In this approach, we utilized spaCy, a natural language processing (NLP) library, for Named Entity Recognition (NER). The code identifies entities such as names, dates, locations, and organizations within the dataset and replaces them with generic labels, effectively anonymizing personally identifiable information (PII) without resorting to regular expressions.

Limitations of NLP(spaCy)-based PII identification:
*  Accuracy depends on NER precision, varying with dataset nuances.
*   Limited context understanding may lead to entity misidentification.
*  Effectiveness may be hindered in languages with immature NLP support.

**4. Deep Learning (Custom NLP Model)**


To accomplish this task, we require a dataset with clear labels to train a deep learning model designed to identify Personally Identifiable Information (PII) within log files.

One effective method involves leveraging a Natural Language Processing (NLP) model built upon Recurrent Neural Networks (RNNs). This approach can significantly enhance the precision of PII detection.

Below is an illustrative model demonstrating this approach:

https://www.kaggle.com/code/awsaf49/pii-data-detection-kerasnlp-starter-notebook/notebook#%F0%9F%93%A9-|-Submission

In [17]:
import json

# Function to tokenize a line into words
def tokenize_line(line):
    return line.split()

# Function to read the log file, tokenize each line, and write to JSON
def log_to_json(log_file, json_file):
    # Open log file for reading
    with open(log_file, 'r') as infile:
        # Initialize list to store JSON objects
        json_data = []

        # Process each line in the log file
        for line_number, line in enumerate(infile):
            # Tokenize the line into words
            words = tokenize_line(line)

            # Determine trailing whitespace
            trailing_whitespace = [c.isspace() for c in line.strip()]

            # Create JSON object with tokenized words and trailing whitespace
            json_obj = {
                "line": line_number,  # Assign unique identifier to each line
                "full_text": line.strip(),  # Store original line
                "tokens": words,         # Store tokenized words
                "trailing_whitespace": trailing_whitespace  # Store trailing whitespace
            }

            # Append JSON object to list
            json_data.append(json_obj)

    # Write list of JSON objects to JSON file
    with open(json_file, 'w') as outfile:
        json.dump(json_data, outfile, indent=4)

# Example usage:
log_file = 'Linux_2k.log'
json_file = 'tokenized_data.json'
log_to_json(log_file, json_file)


In [18]:
import spacy
from spacy.tokens import DocBin


nlp = spacy.load("en_core_web_sm")

# Function to label tokens as PII or not
def label_tokens(data):
    labeled_data = []
    for log in data:
        tokens = log["tokens"]
        text = " ".join(tokens)
        doc = nlp(text)
        pii_labels = [True if ent.label_ == "MISC" else False for ent in doc.ents]
        labeled_data.append({"line": log["line"], "tokens": tokens, "pii_labels": pii_labels})
    return labeled_data

# Write labeled data to output file
def write_labeled_data(data, output_file):
    doc_bin = DocBin()
    for log in data:
        tokens = log["tokens"]
        pii_labels = log["pii_labels"]
        text = " ".join(tokens)
        doc = nlp.make_doc(text)
        for token, pii_label in zip(doc, pii_labels):
            if pii_label:
                doc.ents = [(token.idx, token.idx + len(token), "MISC")]
        doc_bin.add(doc)
    doc_bin.to_disk(output_file)


data = json.load(open("/content/tokenized_data.json"))

# Label tokens as PII or not
labeled_data = label_tokens(data)

# Write labeled data to output file
output_file = "labeled_data.spacy"
write_labeled_data(labeled_data, output_file)


# Conclusion:

**Q1.** Is it possible to anonymize the dataset using NLP?

**Ans:** NLP techniques can definitely help anonymize datasets. By splitting text into tokens, recognizing named entities, and masking sensitive info, we can keep data private while maintaining its usefulness. Custom rules and synthetic data generation are also handy.
So Yes, NLP is a solid tool for keeping data safe!

**Q2.** Does it ‘successfully’ anonymize?

**Ans:** Absolutely! Custom models tailored for log data can take anonymization to the next level. By leveraging techniques like RNNs, which grasp context better, we can ensure even more accurate anonymization of PII in logs. It's all about fine-tuning for the task at hand!

**Q3.** How easy is it to use NLP?

**Ans:** NLP can be straightforward when using pre-trained models or libraries. They offer easy-to-use interfaces and documentation. However, customization and fine-tuning can make things more complex, requiring a deeper understanding and additional effort.

**Q4.** Does it make sense to use NLP?

**Ans:**
Indeed, NLP stands out as an invaluable tool for detecting PII. Given the dynamic nature of PII and the paramount importance of privacy, accuracy in detection and anonymization is crucial. In this regard, NLP emerges as the optimal choice, offering the capabilities needed to ensure precise identification and effective anonymization of sensitive information.

**Q5.** Are the available libraries good enough?

**Ans:** Yes, in a broad sense, the available libraries for NLP perform well. However, for specific tasks like processing log files, they require fine-tuning and customization. For instance, they may overlook the importance of handling IP addresses in logs, which is a critical aspect. Therefore, for such specialized tasks and others like them, custom solutions or the development of custom libraries may be necessary.